## 3.此程式:
最後一步爬取每個評論並儲存到csv中

## Package loading

In [3]:
import csv #This package lets us save data to a csv file
from selenium import webdriver #The Selenium package we'll need
import time #This package lets us pause execution for a bit
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
from pathlib import Path

## Web driver setting

In [4]:
def web_driver_setting():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-images')
    options.add_argument('--disable-gpu')
    options.add_argument("--disable-javascript")
    # 無痕模式
    options.add_argument('--incognito')
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    driver.maximize_window()
    
    return driver

## Scraping function define

In [5]:
# 獲得, locationId
def get_ids_from_hotel_url(url):

  url = url.split('-')

  geo = url[1]

  loc = url[2]

  return (int(geo[1:]), int(loc[1:]))

In [6]:
# driver.get("https://www.tripadvisor.com/Hotel_Review-g60763-d8732904-Reviews-Edge_Hotel-New_York_City_New_York.html")

# time.sleep(2)
# driver.find_element(By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]").click()
# blocks = driver.find_elements(By.XPATH, '//div[@data-test-target="HR_CC_CARD"]')
# for block in blocks:
#     # Grab date of stay
#     # date_of_stay = block.find_element(By.XPATH, './/span[@class="teHYY _R Me S4 H3"]').text.split(":")[1].strip()
            
#     # Grab trip and date type
#     trip_date_element = block.find_elements(By.XPATH, './/div[@class="vTVDc"]//span')
#     # Initialize trip_type and date as 'NULL'
#     trip_type = 'NULL'
#     date_of_stay = 'NULL'
#     # Loop through each span element and check for the class
#     for span_element in trip_date_element:
#         if 'TDKzw _R Me' in span_element.get_attribute('class'):
#             trip_type = block.find_element(By.XPATH, './/span[@class="TDKzw _R Me"]').text.split(":")[1].strip()
#             break  # Found the trip type, no need to continue searching
#     print(trip_type)
    
#     for span_element in trip_date_element:
#         if 'teHYY _R Me S4 H3' in span_element.get_attribute('class'):
#             date_of_stay = block.find_element(By.XPATH, './/span[@class="teHYY _R Me S4 H3"]').text.split(":")[1].strip()
#     print(date_of_stay)

In [7]:
# driver = web_driver_setting()
# driver.get('https://www.tripadvisor.com/Hotel_Review-g60763-d1733337-Reviews-Sutton_Court_Hotel_Residences-New_York_City_New_York.html')

In [8]:
# driver.get('https://www.tripadvisor.com/Hotel_Review-g60763-d1733337-Reviews-Sutton_Court_Hotel_Residences-New_York_City_New_York.html')
# time.sleep(2)
# reviews_numbers_element = driver.find_elements(By.XPATH, '//label[@class="Qukvo Vm _S"]')
# for r in reviews_numbers_element:
#     language = r.find_element(By.XPATH, './/span[@class="ZmySZ q"]').text
#     print(language)
#     if language == 'English':
#         reviews_numbers_ele = r.find_element(By.XPATH, './/span[@class="POjZy"]').text
#         reviews_numbers = int(reviews_numbers_ele.strip('()'))
#         break
# print(reviews_numbers)

In [9]:
# driver = web_driver_setting()
# driver.get('https://www.tripadvisor.com/Hotel_Review-g60763-d8732904-Reviews-Edge_Hotel-New_York_City_New_York.html')
# time.sleep(2)
# blocks = driver.find_elements(By.XPATH, '//div[@data-test-target="HR_CC_CARD"]')
# for b in blocks:
#     try:
#         rhv = b.find_element(By.XPATH, './/span[@class="hVSKz S2 H2 Ch sJlxi"]')
        
#         review_helpful_votes = rhv.text.split(' ')[0]
#     except: 
#         review_helpful_votes = 0
#     print(review_helpful_votes)

In [10]:
# time.sleep(2)
# language_numbers_element = driver.find_elements(By.XPATH, '//li[@class="ui_radio XpoVm"]')
# for r in language_numbers_element:
#     language = r.find_element(By.XPATH, './/span[@class="ZmySZ q"]').text
#     print(language)
#     if language == 'English':
#         reviews_numbers_ele = r.find_element(By.XPATH, './/span[@class="POjZy"]').text
#         reviews_numbers = int(reviews_numbers_ele.strip('()'))
#         break
# print(reviews_numbers)

In [11]:
def get_hotel_review(driver, hotel_url):
    #先取得LocationId和GeoID
    geo, loc = get_ids_from_hotel_url(hotel_url)
    
    # 先和目標網頁建立連線
    driver.get(hotel_url)
    time.sleep(2)
    try:
        # 爬取評論數量 
        reviews_numbers_element = driver.find_elements(By.XPATH, '//label[@class="Qukvo Vm _S"]')
        for r in reviews_numbers_element:
            language = r.find_element(By.XPATH, './/span[@class="ZmySZ q"]').text
            if language == 'English':
                reviews_numbers_ele = r.find_element(By.XPATH, './/span[@class="POjZy"]').text
                reviews_numbers_text = reviews_numbers_ele.strip('()')
                reviews_numbers = re.sub('[^0-9]', '', reviews_numbers_text)
                break
            
        # 設定要爬取的頁面數量
        if int(reviews_numbers)%10 == 0:
            pages_to_scrape = int(reviews_numbers)/10
        else:
            pages_to_scrape = int(int(reviews_numbers)/10)+1
        # 若評論數夠多，限制在10頁，最多爬100筆
        if(pages_to_scrape > 10):
            pages_to_scrape = 10
    except Exception as e:
        print(str(e))
        pages_to_scrape = 0

    #儲存loop 跑的每筆資料
    row_data = []

    for i in range(pages_to_scrape):
        # give the DOM time to load
        time.sleep(2)

        # Click the "expand review" link to reveal the entire review.
        driver.find_element(By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]").click()

        # 尋找所有整個評論的評論區塊元素，return list
        # profiles = driver.find_elements(By.XPATH, '//div[@class="IHLTq _Z o"]')
        # containers = driver.find_elements(By.XPATH, '//div[@data-test-target="reviews-tab"]//div[@data-reviewid]')
        # print(containers)
        blocks = driver.find_elements(By.XPATH, '//div[@data-test-target="HR_CC_CARD"]')

        for block in blocks:
            # Grab review ID
            review_id = block.find_element(By.XPATH, ".//div[@class='WAllg _T']").get_attribute("data-reviewid")
            
            # Grab date
            date = block.find_element(By.XPATH, './/div[@class="cRVSd"]/span')
            md = date.text.split("review")[1].strip()

            # Grab contributions and helpful vote
            element = block.find_element(By.XPATH, './/div[@class="MziKN"]')
            text = element.text

            # 使用正則表達式來提取數字部分
            contributions_match = re.search(r'(\d+)\s*contributions', text)
            helpful_vote_match = re.search(r'(\d+)\s*helpful votes', text)

            # 提取 contributions 數字
            if contributions_match:
                contributions = int(contributions_match.group(1))
            else:
                contributions = 0
            
            # 提取 helpful vote 數字
            if helpful_vote_match:
                helpful_vote = int(helpful_vote_match.group(1))
            else:
                helpful_vote = 0

                # 在每個評論區塊元素中尋找評分、標題和評論

            # Grab rating
            rating = block.find_element(By.XPATH, ".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3] # get_attribute("class") mean 

            # Grab title
            title = block.find_element(By.XPATH, ".//div[@data-test-target='review-title']").text

            # Grab review_content 
            review = block.find_element(By.XPATH, './/span[@class="QewHA H4 _a"]/span').text.replace("\n", "  ")
            
            # Grab trip and date type
            trip_date_element = block.find_elements(By.XPATH, './/div[@class="vTVDc"]//span')
            # Initialize trip_type and date as 'NULL'
            trip_type = 'NULL'
            date_of_stay = 'NULL'
            # Loop through each span element and check for the class
            for span_element in trip_date_element:
                if 'TDKzw _R Me' in span_element.get_attribute('class'):
                    trip_type = block.find_element(By.XPATH, './/span[@class="TDKzw _R Me"]').text.split(":")[1].strip()
                    break  # Found the trip type, no need to continue searching
            # print(trip_type)
            
            for span_element in trip_date_element:
                if 'teHYY _R Me S4 H3' in span_element.get_attribute('class'):
                    date_of_stay = block.find_element(By.XPATH, './/span[@class="teHYY _R Me S4 H3"]').text.split(":")[1].strip()
            # print(date_of_stay)
            
            # Grab review helpful vote
            try:
                rhv = block.find_element(By.XPATH, './/span[@class="hVSKz S2 H2 Ch sJlxi"]')
                
                review_helpful_votes = rhv.text.split(' ')[0]
            except: 
                review_helpful_votes = 0
            
            entry = {
                "Hotel_locID" : loc,
                "Hotel_geoID" : geo,
                "Review_id": review_id,
                "Review_Date": md,
                "Date_of_stay": date_of_stay,
                "Reviewer_Contributions": contributions,
                "Reviewer_helpful_vote": helpful_vote,
                "Review_Rating": rating,
                "Trip_type": trip_type,
                "Review_helpful_votes" : review_helpful_votes,
                "Title": title,
                "Review": review
            }
            row_data.append(entry)
        
        # 翻頁至下一頁，但最後一頁不翻頁
        if i != pages_to_scrape-1:
            driver.find_element(By.XPATH, '//a[@class="ui_button nav next primary "]').click()

    return row_data

    # 完成所有頁面爬取後，關閉瀏覽器
    driver.quit()

## Run function

In [12]:
# # 讀檔
# csvFile = open('TA_review_0808.csv', 'a', encoding="utf-8")
# csvWriter = csv.writer(csvFile)

In [13]:
# 讀取hotel review url list
def get_hotels_list(start, end):
    hotel_data_read=np.load('/home/hao/Code/variable/hotel_data_la_store.npy', allow_pickle=True)
    hotel_data=hotel_data_read.tolist()
    hotels_list = []
    for h in hotel_data:
        hotels_list.append('https://www.tripadvisor.com'+h['Url'])
        
    # 一次只處理一個範圍內的飯店
    # start - end(不含end)
        
    hotels_list = hotels_list[start:end]
    return hotels_list

In [12]:
# get_hotels_list(0, 5)

In [13]:
# 隨機抽取五筆
# hotel_data_random = random.sample(hotel_data, 3)

In [15]:
def main(start, end): # parallel running
    df = pd.DataFrame(columns=["Hotel_locID", "Hotel_geoID", "Review_id", "Review_Date", "Date_of_stay", "Reviewer_Contributions", "Reviewer_helpful_vote", "Review_Rating", "Trip_type", "Review_helpful_votes"  ,"Title", "Review"])

    hotels_list = get_hotels_list(start, end)
    print("Hotel list length is "+str(len(hotels_list)))
    
    driver = web_driver_setting()
    # traversal all hotels' url
    for hotel_url in tqdm(hotels_list):
        try:
            data = get_hotel_review(driver, hotel_url)
            df = pd.concat([df, pd.DataFrame(data)])
        except Exception as e:
            f = Path('error_info_TA_review.txt').open('a')
            f.write("Hotel URL: "+ hotel_url + "\n" + "Error info: " + str(e) + "\n")
            f.close()
            print(e)
            continue
        
    f = Path('error_info_TA_review.txt').open('a')
    ftime = time.strftime("%Y-%m-%d %I:%M:%S %p", time.localtime())
    f.write("FINISH!!!!!!!!!! " + ftime + "\n")
    f.close()
    # df.to_csv(f"result_csv/TA_review_{start}_to_{end}.csv", index=False)
    df.to_csv(f"result_csv/TA_review_LA_{ftime}.csv", index=False)

In [18]:
# hotel_data_read=np.load('/home/hao/Code/variable/hotel_data_store.npy', allow_pickle=True)
# hotel_data=hotel_data_read.tolist()
# hotels_list = []
# for h in hotel_data:
#     hotels_list.append('https://www.tripadvisor.com'+h['Url'])
    
# len(hotels_list)

In [12]:
# 一次只處理一個範圍內的飯店
# start - end(不含end), main(start, end(不含))
main(0, 503)

Hotel list length is 2


100%|██████████| 2/2 [03:12<00:00, 96.28s/it]
